<a href="https://colab.research.google.com/github/kircherlab/MPRAsnakeflow_tutorial/blob/development/tutorial_association.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MPRAsnakeflow association tutorial

In [1]:
# import matplotlib.pyplot as plt
# import os
# import cv2
# from google.colab.patches import cv2_imshow

## Pre-requirements

### Docker
Docker using udocker in colab from github.com/drengskapur/docker-in-colab.

In [2]:
# Copyright 2024 Drengskapur
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# @title {display-mode:"form"}
# @markdown <br/><br/><center><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/docker.svg" height="150"><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/colab.svg" height="150"></center><br/>
# @markdown <center><h1>Docker in Colab</h1></center><center>github.com/drengskapur/docker-in-colab<br/><br/><br/><b>udocker("run hello-world")</b></center><br/>
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

Running docker locally



### snakemake

snakemake v8.16.0 using the official docker image.

In [3]:
udocker("pull visze/mprasnakeflow_tutorial:0.1.0")
udocker("create --name=mprasnakeflow_tutorial visze/mprasnakeflow_tutorial:0.1.0")

user@pc$ docker pull visze/mprasnakeflow_tutorial:0.1.0
docker: 'pull visze/mprasnakeflow_tutorial:0.1.0' is not a docker command.
See 'docker --help'
user@pc$ docker create --name=mprasnakeflow_tutorial visze/mprasnakeflow_tutorial:0.1.0
docker: 'create --name=mprasnakeflow_tutorial visze/mprasnakeflow_tutorial:0.1.0' is not a docker command.
See 'docker --help'


Try to run `snakemake --version` in docker:

In [4]:
udocker("run mprasnakeflow_tutorial snakemake --version")

user@pc$ udocker run mprasnakeflow_tutorial snakemake --version
su: user user does not exist or the user entry does not contain all the required fields


### Test data

The data used in this tutorial is present in the github repository. We will dowload it using git so that it is also present within a colab environment.

In [5]:
%%bash
git clone https://github.com/kircherlab/MPRAsnakeflow_tutorial.git
cd MPRAsnakeflow_tutorial
git checkout development

fatal: destination path 'MPRAsnakeflow_tutorial' already exists and is not an empty directory.


Already on 'development'


Your branch is behind 'origin/development' by 1 commit, and can be fast-forwarded.
  (use "git pull" to update your local branch)


## Run MPRAsnakeflow




### Snakemake

Before we come to more MPRAsnakeflow pipeline specific things we quickly want to introduce the snakemake workflow management system. A workflow management system, also like nextflow or cwl, is a tool to create reproducible and scalable data analyses. Snakemake workflows are described via a human readable, Python based language. They can be seamlessly scaled to server, cluster, grid and cloud environments, without the need to modify the workflow definition. Finally, Snakemake workflows can entail a description of required software, which will be automatically deployed to any execution environment.

Each step in snakemake is defined as a `rule`. Which has a `input`, `output` and a mostly `shell` command. The `input` and `output` are files that are used as input and output for the rule. The `shell` command is the command that is executed to create the output file from the input file. There are other commands like `conda`, `params`, `log`, `threads`, `resources` that can be used to define the rule.

E.g. here is an example rule `assignment_check_design` of MPRAsnakeflow, which checks if the design file is correct (no duplicated header, no `[` or `]` within headers and no duplicates sequences (sense and antisense). The rule takes our design file defined in the config (see later) and a script as input. When it is completed it creates the file `results/assignment/{assignment}/design_check.done` and other rules will only start running when this file is present and the check is completed. `{assignment}` is a wildcard and the name of the assignment will be used here (defined in the config file). This allows us to define multiple assignments within the config file an run them in parallel. In addition the design file is copied here: `results/assignment/{assignment}/reference/reference.fa` so that further mapping rules can create their mapping reference files. You see the `params` command is a bit lengthy but we needed some conditions, based on the flexibility of the user input. But finally the `shell` command looks very clean.

```python
rule assignment_check_design:
    """
    Check if the design file is correct and no duplicated sequences are present (FW and reverse).
    Also check if no duplicated headers and no illegal characters in header.
    """
    conda:
        "../envs/python3.yaml"
    input:
        design=lambda wc: config["assignments"][wc.assignment]["design_file"],
        script=getScript("assignment/check_design_file.py"),
    output:
        temp("results/assignment/{assignment}/reference/reference.fa.fxi"),
        touch("results/assignment/{assignment}/design_check.done"),
        ref="results/assignment/{assignment}/reference/reference.fa",
    params:
        start=lambda wc: (
            config["assignments"][wc.assignment]["alignment_tool"]["configs"][
                "alignment_start"
            ]
            if config["assignments"][wc.assignment]["alignment_tool"]["tool"]
            == "exact"
            else config["assignments"][wc.assignment]["alignment_tool"]["configs"][
                "alignment_start"
            ]["max"]
        ),
        length=lambda wc: (
            config["assignments"][wc.assignment]["alignment_tool"]["configs"][
                "sequence_length"
            ]
            if config["assignments"][wc.assignment]["alignment_tool"]["tool"]
            == "exact"
            else config["assignments"][wc.assignment]["alignment_tool"]["configs"][
                "sequence_length"
            ]["min"]
        ),
        fast_check=lambda wc: (
            "--fast-dict"
            if config["assignments"][wc.assignment]["design_check"]["fast"]
            else "--slow-string-search"
        ),
        check_sequence_collitions=lambda wc: (
            "--perform-sequence-check"
            if config["assignments"][wc.assignment]["design_check"][
                "sequence_collitions"
            ]
            else "--skip-sequence-check"
        ),
    log:
        log=temp("results/logs/assignment/check_design.{assignment}.log"),
        err="results/assignment/{assignment}/design_check.err",
    shell:
        """
        trap "cat {log.err}" ERR
        cp {input.design} {output.ref}
        python {input.script} --input {output.ref} \
        --start {params.start} --length {params.length} \
        {params.fast_check} {params.check_sequence_collitions} > {log.log} 2> {log.err};
        """
```


When running snakemake you need to specify where your worflow is, basically the main `Snakefile` of it. By default, Snakemake will search for ‘Snakefile’, ‘snakefile’, ‘workflow/Snakefile’, ‘workflow/snakefile’ beneath the current working directory, in this order. Then you do not have to use this command. But practically your workflow, here MPRAsnakeflow, might live somewhere else and you want to run the workflow in a different directory where you want to produce results. Then you need to specify the `--snakefile` parameter to the MPRAsnakeflow file in `workflow/Snakefile`. Otherwise you ahve to check/copy the workflow in every "run" rirectory which is not practically. Usually you start snakemake in the directory where you want to produce results.

### The config file

The heart of your workflow run is the config file. You can specify it via `--configfile`. It is in `yaml` or `json` format. It contains all the parameters and input files that are used in the workflow.

The config file has many parameters. But most of them have a default well, based on best practice and you don't have to specify them. But you can overwrite them if you want. Config files are first validated using json schema. If your config file is not correct MPRAsnakeflow will tell you what is missing or wrong and stop. Naming, parameters etc. might change from MPRAsnakeflow version and you might have to update your config file if you run a new version of MPRAsnakeflow. An extensive documentation of the config file can be found in the [MPRAsnakeflow documentation](https://mprasnakeflow.readthedocs.io/en/development/config.html).

Here is the config file that we will use for running the association tutorial.



```yaml
---
# global:
#   assignments:
#     split_number: 30
assignments:
  assignMPRAworkshop:
    bc_length: 15
    alignment_tool:
        tool: bwa
        configs:
          sequence_length:
            min: 195
            max: 205
          alignment_start:
            min: 14
            max: 16
    FW:
      - example_data/assignment/assoc_fwd.fastq.gz
    BC:
      - example_data/assignment/assoc_bc.fastq.gz
    REV:
      - example_data/assignment/assoc_rev.fastq.gz
    design_file: example_data/design/workshop_design.fa
    configs:
      default: {}
```

Because we run the assignment workflow we start with `assignment`. The a list with possible names of assignments will follow. Here we have just one: `assignMPRAworkshop`. The whole config file has the minimal information, which is needed by MPRAsnakeflow. This is the barcode length `bc_length`, the alignment tool `alignment_tool` for mapping reads to the design file (here we use `bwa` but other options are possible) and the configuration of the tool. The forward reads `FW`, the barcode reads `BC`, the reverse reads `REV` and the design file `design_file`. Then we can define multiple configs `configs`, basically how we filter barcodes later. E.g. a minimal support of barcodes per oligo `min_support` and a fraction of assigned barcodes to an oligo `fraction`. In our tutorial example we just have one config named `default` and we will set the default parameters by defining `degault: {}`. This is `min_support: 3` and `fraction: 0.75`. But the option to have multiple settings allows us to create multiple BC to oligo mappings based on different strategies. The worflow will then only run the last part multiple times with different settings avoiding the computational extensive mapping approach.

### MPRAsnakeflow tutorial profile

Snakemake can make use of profiles to pre-configure command-line-interfaces as well as ressources and number of threads as default or per rule. A profile has to be stored (on Linux) in the folder `$HOME/.config/snakemake` or `/etc/xdg/snakemake`. It can be activated by using the `--profile` option in the snakemake command. In addition you can set an environment variable `SNAKEMAKE_PROFILE`, e.g. by specifying export `SNAKEMAKE_PROFILE=myprofile` in your `~/.bashrc` to use the profile by default. In addition a specific workflow profile can be set using `--workflow-profile path_to_your_profile`. E.g. we pre-defined a [workflow profile for MPRAsnakeflow](https://github.com/kircherlab/MPRAsnakeflow/blob/development/profiles/default/config.yaml) adding ressources to each rule with some additional commands for the [snakemake excecutor plugin slum](https://snakemake.github.io/snakemake-plugin-catalog/plugins/executor/slurm.html).

For more information on snakemake using profiles please have a look [snakemke profile documentation](https://snakemake.readthedocs.io/en/stable/executing/cli.html#profiles)

For the MPRAsnakeflow tutorial we will use the profile `mprasnakeflow` which is stored in the folder `/etc/xdg/snakemake` within the container and the `SNAKEMAKE_PROFILE=mprasnakeflow` is already set. Have a look at the variable and the profile:

In [6]:
udocker("run \
        mprasnakeflow_tutorial \
        cat /etc/xdg/snakemake/mprasnakeflow/config.yaml")

user@pc$ udocker run         mprasnakeflow_tutorial         echo $SNAKEMAKE_PROFILE &&         cat /etc/xdg/snakemake/mprasnakeflow/config.yaml
su: user user does not exist or the user entry does not contain all the required fields


You see that we already defined where the MPRASsnkeflow `Snakefile` is and `--snakefile` is not necessary. Also all software packages are installed alrady in the container using conda and we enable this software deployment method and tell snakemake where the conda environments live (the default will be your exceuction folder under `.snamemake/conda`). Also the `--cores/-c` option is not necessary anymore and snakemake just needs your configuration file of the workflow using the command `--configfile`

### Run the workflow

First we run a dry-run to see what will be executed using `-n` and `--quiet rules` to see only the overview of excecuted rules and not each rule command individually. The most important part is setting the config file via `-configfile assignment_config.yaml`

In [7]:
# MPRAsnakeflow_tutorial must be accessable and writable for all (and docker)
!chmod ugo+rwx MPRAsnakeflow_tutorial
# Run MPRAsnakefile
udocker("run \
        -v=${PWD}/MPRAsnakeflow_tutorial:/data/run \
        --workdir /data/run \
        mprasnakeflow_tutorial \
        snakemake --configfile config_assignment.yaml \
        -n --quiet rules")

user@pc$ udocker run         -v=${PWD}/MPRAsnakeflow_tutorial:/data/run         --workdir /data/run         mprasnakeflow_tutorial       snakemake --configfile config_assignment.yaml       -n --quiet rules
su: user user does not exist or the user entry does not contain all the required fields


You see a list of rules and how often this rule is executed debending on the input file. E.g. it is possible to you input fastq file into `n` via the rule `assignment_fastq_split` using the option `split_number` in the config file (it is commented out right now). Then rules afterwards, like `assignment_getBCs` will be executed `n` times. Tis behaviour is nice for parallelisation especially on a HPC system where you have multiple nodes available. 

A short summary of each rule is in the [MPRAsnakeflow documentation](https://mprasnakeflow.readthedocs.io/en/latest/assignment.html#rules).

Now let's run snakemake using 10 cores (BWA can be parallized). When you add the `-p` flag you will see the exact code that is executed in each rule. But we ommit this here beacuse it is a lot of output.

In [8]:
udocker("run \
        -v=${PWD}/MPRAsnakeflow_tutorial:/data/run \
        --workdir /data/run \
        mprasnakeflow_tutorial \
        snakemake --configfile config_assignment.yaml \
        -c 10")

user@pc$ udocker run   -v=${PWD}/work:/data/run   --workdir /data/run   mprasnakeflow   snakemake --configfile assignment_config.yaml   -c 10 --quiet rules
su: user user does not exist or the user entry does not contain all the required fields
